In [1]:
""" Zonal statistics for basin demand. Export in table format.
-------------------------------------------------------------------------------
Demand data is provided in volumes (millionm3) at 5min resolution. 
For further analysis the 30s rasterized zones of hydrobasin level 6 will be 
used.

Steps:
    - Convert to flux at 5min.
    - (under the hood) Convert to flux at 30s.
    - Zonal stats for flux at 30s.


Author: Rutger Hofste
Date: 20180422
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    TESTING (boolean) : Testing mode. Uses a smaller geography if enabled.
    
    SCRIPT_NAME (string) : Script name.
    EE_INPUT_ZONES_PATH (string) : earthengine input path for zones.
    EE_INPUT_VALUES_PATH (string) : earthengine input path for value images.
    INPUT_VERSION_ZONES (integer) : input version for zones images.
    INPUT_VERSION_VALUES (integer) : input version for value images.
    OUTPUT_VERSION (integer) : output version. 
    EXTRA_PROPERTIES (dictionary) : Extra properties to store in the resulting
        pandas dataframe. 
    

Returns:


"""

# Input Parameters
TESTING = 0
SCRIPT_NAME = "Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01"

EE_INPUT_ZONES_PATH = "projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V01"
EE_INPUT_VALUES_PATH = "projects/WRI-Aquaduct/PCRGlobWB20V09"
INPUT_VERSION_ZONES = 4

OUTPUT_VERSION = 1

SPATIAL_RESOLUTIONS = ["30s"]
PFAF_LEVELS = [6]

SECTORS = ["PDom","PInd","PIrr","PLiv"]
DEMAND_TYPES = ["WW","WN"]
TEMPORAL_RESOLUTIONS = ["year","month"]
REDUCER_NAMES = ["mean"]

SEPARATOR = "_|-"
SCHEMA =["geographic_range",
         "temporal_range",
         "indicator",
         "temporal_resolution",
         "unit",
         "spatial_resolution",
         "temporal_range_min",
         "temporal_range_max"]

EXTRA_PROPERTIES = {"output_version":OUTPUT_VERSION,
                    "script_used":SCRIPT_NAME,
                   }


# Output Parameters
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)


print("Input ee zones: " +  EE_INPUT_ZONES_PATH +
      "\nInput ee values: " + EE_INPUT_VALUES_PATH +
      "\nOutput gcs: " + gcs_output_path)


Input ee zones: projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V01
Input ee values: projects/WRI-Aquaduct/PCRGlobWB20V09
Output gcs: gs://aqueduct30_v01/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/


In [2]:
import time, datetime, sys, logging
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D24 UTC 08:34


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
# Imports
import pandas as pd
from datetime import timedelta
import os
import ee
import aqueduct3

ee.Initialize()

In [4]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler = logging.FileHandler("./logs/{}.log".format(SCRIPT_NAME))
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [5]:
#!rm -r {ec2_output_path}
#!mkdir -p {ec2_output_path}

In [8]:
def post_process_results(result_list,function_properties,extra_properties=EXTRA_PROPERTIES):
    """Client side function to convert results of reduceRegion to pandas dataframe.
    -------------------------------------------------------------------------------
    
    Adds additional properties. The script is client side for convenience reasons.
    A more robust and fast approach would be to add the extra_properties to the 
    server side dictionary.
    
    Args:
        result_list (ee.List) : List of dictionaries. Result from reduceRegion
        function_properties (dictionary) : Additional properties used in the 
            reduceRegion function call.
        extra_properties (dictionary) : Additional properties set at global level. 
    
    Returns:
        df (pd.DataFrame) : Pandas dataframe with extra properties.
    
    
    """
    extra_properties = {**function_properties, **EXTRA_PROPERTIES}
    result_list_clientside = result_list.getInfo()
    df = pd.DataFrame(result_list_clientside)
    df = df.assign(**extra_properties)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df  


def main():
    geometry = aqueduct3.earthengine.get_global_geometry(TESTING)
    i_processed = 0

    if TESTING:
        sectors = ["PDom"]
        demand_types = ["WW"]
        temporal_resolutions = ["year"]
        reducer_names = ["mean"]
        spatial_resolutions = ["30s"]
        pfaf_levels = [6]
    else:
        sectors = SECTORS
        demand_types = DEMAND_TYPES
        temporal_resolutions = TEMPORAL_RESOLUTIONS
        reducer_names = REDUCER_NAMES
        spatial_resolutions = SPATIAL_RESOLUTIONS
        pfaf_levels = PFAF_LEVELS

    start_time = time.time()
    for reducer_name in reducer_names:
        reducer = aqueduct3.earthengine.get_grouped_reducer(reducer_name)    
        for spatial_resolution in spatial_resolutions:
            crs_transform = aqueduct3.earthengine.get_crs_transform(spatial_resolution)
            for pfaf_level in pfaf_levels:            
                for sector in sectors:
                    for demand_type in demand_types:
                        for temporal_resolution in temporal_resolutions:
                            print(reducer_name,spatial_resolution,pfaf_level,sector,demand_type,temporal_resolution)

                            i_zones_asset_id = "{}/hybas_lev{:02.0f}_v1c_merged_fiona_{}_V{:02.0f}".format(EE_INPUT_ZONES_PATH,pfaf_level,spatial_resolution,INPUT_VERSION_ZONES)
                            ic_values_input_asset_id = "{}/global_historical_{}{}_{}_m_5min_1960_2014".format(EE_INPUT_VALUES_PATH,sector,demand_type,temporal_resolution)
                            df = aqueduct3.earthengine.get_df_from_ic(ic_values_input_asset_id)

                            if TESTING:
                                df = df[1:3]
                            else:
                                pass

                            for index, row in df.iterrows():
                                i_processed = i_processed + 1
                                elapsed_time = time.time() - start_time
                                i_values_input_asset_id = row["input_image_asset_id"]
                                # Add an artificial extension to allow the function to run. 
                                # consider updating the split_key function to handle cases without an extension.
                                i_values_input_asset_id_extenstion = i_values_input_asset_id + ".ee_image"
                                dictje = aqueduct3.split_key(i_values_input_asset_id_extenstion,SCHEMA,SEPARATOR)
                                output_file_name = "{}_reduced_{:02.0f}_{}_{}".format(dictje["file_name"],pfaf_level,spatial_resolution,reducer_name)
                                output_file_path_pkl = "{}/{}.pkl".format(ec2_output_path,output_file_name)
                                output_file_path_csv = "{}/{}.csv".format(ec2_output_path,output_file_name)
                                
                                if os.path.isfile(output_file_path_pkl):
                                    message = "Index {:02.2f}, Skipping: {} Elapsed: {} Asset: {}".format(float(index),i_processed,str(timedelta(seconds=elapsed_time)),i_values_input_asset_id)
                                    logger.debug(message)
                                else:
                                    message = "Index {:02.2f}, Processed: {} Elapsed: {} Asset: {}".format(float(index),i_processed,str(timedelta(seconds=elapsed_time)),i_values_input_asset_id)
                                    print(message)
                                    logger.debug(message)
                                                     
                                    i_values = ee.Image(i_values_input_asset_id)
                                    total_image = ee.Image(i_values_input_asset_id).addBands(ee.Image(i_zones_asset_id))
                                    result_list = total_image.reduceRegion(geometry = geometry,
                                                    reducer= reducer,
                                                    crsTransform = crs_transform,
                                                    maxPixels=1e10
                                                    ).get("groups")

                                    function_properties = {"zones_pfaf_level":pfaf_level,
                                                           "zones_spatial_resolution":spatial_resolution,
                                                           "reducer":reducer_name,
                                                           "zones_image_asset_id":i_zones_asset_id}


                                    function_properties = {**function_properties, **dictje}
                                    df = post_process_results(result_list,function_properties)


                                    df.to_pickle(output_file_path_pkl)
                                    #df.to_csv(output_file_path_csv,encoding='utf-8')
 
                                    

if __name__ == "__main__":
    main()


mean 30s 6 PDom WW year
mean 30s 6 PDom WW month
mean 30s 6 PDom WN year
mean 30s 6 PDom WN month
mean 30s 6 PInd WW year
mean 30s 6 PInd WW month
mean 30s 6 PInd WN year
mean 30s 6 PInd WN month
mean 30s 6 PIrr WW year
mean 30s 6 PIrr WW month
mean 30s 6 PIrr WN year
mean 30s 6 PIrr WN month
mean 30s 6 PLiv WW year
mean 30s 6 PLiv WW month
Index 529.00, Processed: 4875 Elapsed: 0:00:30.812168 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I529Y2004M02
Index 530.00, Processed: 4876 Elapsed: 0:00:44.476017 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I530Y2004M03
Index 531.00, Processed: 4877 Elapsed: 0:00:57.692149 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I531Y2004M04
Index 532.00, Processed: 4878 Elapse

Index 567.00, Processed: 4913 Elapsed: 0:08:45.427660 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I567Y2007M04
Index 568.00, Processed: 4914 Elapsed: 0:08:58.324120 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I568Y2007M05
Index 569.00, Processed: 4915 Elapsed: 0:09:11.051423 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I569Y2007M06
Index 570.00, Processed: 4916 Elapsed: 0:09:23.278991 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I570Y2007M07
Index 571.00, Processed: 4917 Elapsed: 0:09:36.183914 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_m

Index 607.00, Processed: 4953 Elapsed: 0:17:16.153815 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I607Y2010M08
Index 608.00, Processed: 4954 Elapsed: 0:17:29.804690 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I608Y2010M09
Index 609.00, Processed: 4955 Elapsed: 0:17:42.775852 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I609Y2010M10
Index 610.00, Processed: 4956 Elapsed: 0:17:55.859420 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I610Y2010M11
Index 611.00, Processed: 4957 Elapsed: 0:18:08.714795 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_m

Index 647.00, Processed: 4993 Elapsed: 0:25:51.692176 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I647Y2013M12
Index 648.00, Processed: 4994 Elapsed: 0:26:04.746660 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I648Y2014M01
Index 649.00, Processed: 4995 Elapsed: 0:26:17.284008 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I649Y2014M02
Index 650.00, Processed: 4996 Elapsed: 0:26:29.871803 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_month_m_5min_1960_2014_I650Y2014M03
Index 651.00, Processed: 4997 Elapsed: 0:26:42.366110 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWW_month_m_5min_1960_2014/global_historical_PLivWW_m

Index 27.00, Processed: 5033 Elapsed: 0:34:24.776715 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_year_m_5min_1960_2014/global_historical_PLivWN_year_m_5min_1960_2014_I027Y1987M12
Index 28.00, Processed: 5034 Elapsed: 0:34:37.877495 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_year_m_5min_1960_2014/global_historical_PLivWN_year_m_5min_1960_2014_I028Y1988M12
Index 29.00, Processed: 5035 Elapsed: 0:34:50.652190 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_year_m_5min_1960_2014/global_historical_PLivWN_year_m_5min_1960_2014_I029Y1989M12
Index 30.00, Processed: 5036 Elapsed: 0:35:03.123485 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_year_m_5min_1960_2014/global_historical_PLivWN_year_m_5min_1960_2014_I030Y1990M12
Index 31.00, Processed: 5037 Elapsed: 0:35:15.430724 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_year_m_5min_1960_2014/global_historical_PLivWN_year_m_5min_196

...

Index 492.00, Processed: 5553 Elapsed: 2:36:56.761276 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I492Y2001M01
Index 493.00, Processed: 5554 Elapsed: 2:37:10.683575 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I493Y2001M02
Index 494.00, Processed: 5555 Elapsed: 2:37:25.013675 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I494Y2001M03
Index 495.00, Processed: 5556 Elapsed: 2:37:38.943118 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I495Y2001M04
Index 496.00, Processed: 5557 Elapsed: 2:37:52.435351 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_m

Index 532.00, Processed: 5593 Elapsed: 2:46:00.226034 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I532Y2004M05
Index 533.00, Processed: 5594 Elapsed: 2:46:13.290585 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I533Y2004M06
Index 534.00, Processed: 5595 Elapsed: 2:46:27.395540 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I534Y2004M07
Index 535.00, Processed: 5596 Elapsed: 2:46:42.424989 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I535Y2004M08
Index 536.00, Processed: 5597 Elapsed: 2:46:56.168437 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_m

Index 572.00, Processed: 5633 Elapsed: 2:55:57.949045 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I572Y2007M09
Index 573.00, Processed: 5634 Elapsed: 2:56:11.543812 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I573Y2007M10
Index 574.00, Processed: 5635 Elapsed: 2:56:25.995682 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I574Y2007M11
Index 575.00, Processed: 5636 Elapsed: 2:56:39.763724 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I575Y2007M12
Index 576.00, Processed: 5637 Elapsed: 2:56:54.829240 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_m

Index 612.00, Processed: 5673 Elapsed: 3:05:53.111512 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I612Y2011M01
Index 613.00, Processed: 5674 Elapsed: 3:06:09.833606 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I613Y2011M02
Index 614.00, Processed: 5675 Elapsed: 3:06:25.698339 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I614Y2011M03
Index 615.00, Processed: 5676 Elapsed: 3:06:40.757930 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I615Y2011M04
Index 616.00, Processed: 5677 Elapsed: 3:06:55.956342 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_m

Index 652.00, Processed: 5713 Elapsed: 3:15:47.990566 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I652Y2014M05
Index 653.00, Processed: 5714 Elapsed: 3:16:01.926659 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I653Y2014M06
Index 654.00, Processed: 5715 Elapsed: 3:16:19.059733 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I654Y2014M07
Index 655.00, Processed: 5716 Elapsed: 3:16:37.232098 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_month_m_5min_1960_2014_I655Y2014M08
Index 656.00, Processed: 5717 Elapsed: 3:16:59.628228 Asset: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_PLivWN_month_m_5min_1960_2014/global_historical_PLivWN_m

In [11]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --exclude='*' --include='*.pkl'

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I001Y1960M02_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I001Y1960M02_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I005Y1960M06_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I005Y1960M06_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I009Y1960M10_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I009Y196

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I030Y1962M07_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I030Y1962M07_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I024Y1962M01_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I024Y1962M01_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I026Y1962M03_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I026Y196

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I047Y1963M12_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I047Y1963M12_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I050Y1964M03_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I050Y1964M03_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I049Y1964M02_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I049Y196

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I074Y1966M03_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I074Y1966M03_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I072Y1966M01_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I072Y1966M01_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I069Y1965M10_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I069Y196

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I095Y1967M12_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I095Y1967M12_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I101Y1968M06_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I101Y1968M06_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I096Y1968M01_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWN_month_m_5min_1960_2014_I096Y196

...

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I563Y2006M12_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I563Y2006M12_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I560Y2006M09_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I560Y2006M09_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I556Y2006M05_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I556Y200

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I587Y2008M12_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I587Y2008M12_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I588Y2009M01_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I588Y2009M01_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I592Y2009M05_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I592Y200

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I614Y2011M03_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I614Y2011M03_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I615Y2011M04_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I615Y2011M04_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I609Y2010M10_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I609Y201

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I637Y2013M02_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I637Y2013M02_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I635Y2012M12_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I635Y2012M12_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I627Y2012M04_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I627Y201

upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I658Y2014M11_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I658Y2014M11_reduced_06_30s_mean.pkl
upload: ../../../../data/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I648Y2014M01_reduced_06_30s_mean.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M04D22_RH_Zonal_Stats_Demand_EE_V01/output_V01/global_historical_PLivWW_month_m_5min_1960_2014_I648Y2014M01_reduced_06_30s_mean.pkl


In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

## Previous runs:  
0:24:15.930678    